In [1]:
print("hello world")

hello world


In [2]:
import requests

url = "https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite"
output_path = "detector.tflite"

response = requests.get(url)
if response.status_code == 200:
    with open(output_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded to {output_path}")
else:
    print(f"Failed to download. HTTP Status Code: {response.status_code}")



Downloaded to detector.tflite


In [ ]:
import yt_dlp
import cv2
import mediapipe as mp

# Function to fetch the direct video URL using yt-dlp
def get_video_url(youtube_url):
    ydl_opts = {
        'quiet': True,
        'format': 'best[ext=mp4]'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        return info['url']

# Initialize MediaPipe solutions
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize Face Detection and Face Mesh for multi-face support
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=10,  # Adjust this value for the maximum number of faces
    min_detection_confidence=0.5
)

# Initialize Pose for multi-person support
pose = mp_pose.Pose(min_detection_confidence=0.5)

# YouTube video URL
youtube_url = "https://www.youtube.com/watch?v=96Y6mc3C1Bg"  # Replace with your video URL

# Fetch the video stream URL
video_url = get_video_url(youtube_url)
print("Video URL:", video_url)

# Open the video stream with OpenCV
cap = cv2.VideoCapture(video_url)

if not cap.isOpened():
    print("Error: Unable to open the YouTube video stream.")
else:
    print("Processing YouTube video. Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or error reading the video stream.")
            break

        # Convert the frame to RGB for MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect faces in the frame
        face_results = face_detection.process(rgb_frame)
        if face_results.detections:
            for detection in face_results.detections:
                # Draw the face bounding box
                mp_drawing.draw_detection(frame, detection)

                # Extract bounding box
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                       int(bboxC.width * iw), int(bboxC.height * ih)

                # Use Face Mesh for detected face
                face_landmarks = face_mesh.process(rgb_frame)
                if face_landmarks.multi_face_landmarks:
                    for face_landmark in face_landmarks.multi_face_landmarks:
                        mp_drawing.draw_landmarks(
                            frame, face_landmark, mp_face_mesh.FACEMESH_TESSELATION,
                            mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style()
                        )

        # Detect pose in the frame
        pose_results = pose.process(rgb_frame)
        if pose_results.pose_landmarks:
            # Draw landmarks for all detected poses
            mp_drawing.draw_landmarks(
                frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

        # Display the processed frame
        cv2.imshow("MediaPipe YouTube Video Processing", frame)

        # Exit when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting video processing.")
            break

# Release resources
cap.release()
cv2.destroyAllWindows()


2024-12-11 02:56:25.970648: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1733914591.957548 2801300 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.14.1), renderer: AMD Radeon Pro 555 OpenGL Engine
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1733914591.983031 2801793 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1733914591.983685 2801300 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.14.1), renderer: AMD Radeon Pro 555 OpenGL Engine
W0000 00:00:1733914591.989641 2801807 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Video URL: https://rr1---sn-n4v7snls.googlevideo.com/videoplayback?expire=1733936193&ei=4W9ZZ8WUGZ3ZsfIP4rOB2Ag&ip=2601%3A644%3A601%3Afa0%3A18f1%3A6d9c%3A9455%3Ae968&id=o-ANjMsFGNi0ReJj4-81jjlSyZNh8HfPuGsaR29dLJguh2&itag=18&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1733914593%2C&mh=KN&mm=31%2C26&mn=sn-n4v7snls%2Csn-a5mekndz&ms=au%2Conr&mv=m&mvi=1&pl=34&rms=au%2Cau&initcwndbps=3852500&bui=AQn3pFQ49B-fCOpk-mkniS0GYUGgd5WNjxuBUOFyzsF_mGv05DRrDZVBwqIr0nwUdoOfDyYYjwX6etUl&vprv=1&svpuc=1&mime=video%2Fmp4&ns=6WJEoQI_Vxi02XvVVuhj1EYQ&rqh=1&cnr=14&ratebypass=yes&dur=5104.187&lmt=1729255083328319&mt=1733914385&fvip=2&fexp=51326932%2C51331020%2C51335594%2C51347747&c=MWEB&sefc=1&txp=5538434&n=BlQJpyhNKywxCg&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%2Ccnr%2Cratebypass%2Cdur%2Clmt&sig=AJfQdSswRAIgbozzjm_lC7kU3VEZTdVas5dk4_HnTBVsvURlbQ6XIF0CIBS4uqfOcDJOXcldSkhyYcMfbwF7i2DlTWggpoXZP2HR&lsparams=met%2Cmh%2Cmm%2Cmn%2Cms%2Cmv%2Cmv

W0000 00:00:1733914598.926110 2801815 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Exiting video processing.


: 

In [ ]:
#old code that just streams without mediapipe facial recognition
# Get the direct video URL
import yt_dlp
import cv2

def get_video_url(youtube_url):
    ydl_opts = {
        'quiet': True,
        'format': 'best[ext=mp4]'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        return info['url']

# Get the direct video URL
youtube_url = "https://www.youtube.com/watch?v=m34ZKJNyxac"
video_url = get_video_url(youtube_url)
print("Video URL:", video_url)

# Open video stream with OpenCV
cap = cv2.VideoCapture(video_url)

if not cap.isOpened():
    print("Error: Unable to open video stream.")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("YouTube Video", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
video_url = get_video_url(youtube_url)
print("Video URL:", video_url)

# Open video stream with OpenCV
cap = cv2.VideoCapture(video_url)

if not cap.isOpened():
    print("Error: Unable to open video stream.")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("YouTube Video", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Video URL: https://rr2---sn-o097znsr.googlevideo.com/videoplayback?expire=1733935790&ei=Tm5ZZ8OCHvr-sfIP1OPhoAg&ip=2601%3A644%3A601%3Afa0%3A18f1%3A6d9c%3A9455%3Ae968&id=o-AM0VWy9id_RhIja_CuAX_7jCOsmuB3nF0M_O12ZHnnTH&itag=18&source=youtube&requiressl=yes&xpc=EgVo2aDSNQ%3D%3D&met=1733914190%2C&mh=Vq&mm=31%2C29&mn=sn-o097znsr%2Csn-n4v7sns7&ms=au%2Crdu&mv=m&mvi=2&pl=34&rms=au%2Cau&initcwndbps=3915000&bui=AQn3pFQKhi7MVAR6il70JWVEwJTRLPZJ7nzHT55pqdE6sdGYDy3fT88Js2ixpJEj8o83imVFwTnhby9Q&vprv=1&svpuc=1&mime=video%2Fmp4&ns=dgnEsjCi-frwcKg6mSObwzwQ&rqh=1&gir=yes&clen=36612911&ratebypass=yes&dur=398.593&lmt=1733879672190924&mt=1733913906&fvip=3&fexp=51326932%2C51331020%2C51335594%2C51347747&c=MWEB&sefc=1&txp=5538434&n=jbygYr9NxynRtw&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%2Cgir%2Cclen%2Cratebypass%2Cdur%2Clmt&sig=AJfQdSswRQIgR_sLg4GdoV2OPdKEWhxKh-0u7jr7-NyGlLR3-mI-5a4CIQCMCLTi1P8C5kClsDZoxqTwkw4zeYhfluOoJaAbPTohlA%3D%3D&lsparams=met%

KeyboardInterrupt: 

: 

In [ ]:
#Todo, need to add cookies for age restricted content